# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import sys
import re
import pickle
import pandas as pd
from sqlalchemy import create_engine
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score
from xgboost import XGBClassifier

In [2]:
# load data from database
engine = create_engine('sqlite:///data/DisasterResponse.db')
df = pd.read_sql("SELECT * FROM messages", engine)

In [3]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
X = ['genre', 'message']
Y = list(set(df.columns)-set(['id', 'message', 'original', 'genre']))

In [5]:
df.loc[df['related'] == 2, 'related'] = 1

In [6]:
for y in Y:
    print(y, df[y].unique())

missing_people [0 1]
related [1 0]
direct_report [0 1]
offer [0 1]
military [0 1]
fire [0 1]
infrastructure_related [0 1]
medical_products [0 1]
request [0 1]
child_alone [0]
hospitals [0 1]
earthquake [0 1]
floods [0 1]
security [0 1]
refugees [0 1]
clothing [0 1]
tools [0 1]
buildings [0 1]
food [0 1]
shelter [0 1]
transport [0 1]
water [0 1]
other_aid [0 1]
aid_centers [0 1]
other_infrastructure [0 1]
other_weather [0 1]
medical_help [0 1]
search_and_rescue [0 1]
money [0 1]
weather_related [0 1]
electricity [0 1]
cold [0 1]
aid_related [0 1]
storm [0 1]
death [0 1]
shops [0 1]


### 2. Write a tokenization function to process your text data

In [7]:
def tokenize(text):
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    # lemmatize andremove stop words
    stop_words = stopwords.words("english")
    tokens = [lemmatizer.lemmatize(word).strip() for word in tokens if word not in stop_words]

    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [8]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [9]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier())),
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X = df['message']
y = df[Y]

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [13]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7f9725e82158>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [14]:
y_pred = pipeline.predict(X_test)

In [15]:
for i in range(len(Y)):
    print('Category: ', y_test.columns[i])
    print(classification_report(pd.DataFrame(y_pred).iloc[:, i], \
        pd.DataFrame(y_test).iloc[:, i]))

Category:  missing_people
              precision    recall  f1-score   support

           0       1.00      0.99      0.99      8651
           1       0.01      1.00      0.02         1

    accuracy                           0.99      8652
   macro avg       0.50      0.99      0.51      8652
weighted avg       1.00      0.99      0.99      8652

Category:  related
              precision    recall  f1-score   support

           0       0.39      0.70      0.50      1128
           1       0.95      0.83      0.89      7524

    accuracy                           0.82      8652
   macro avg       0.67      0.77      0.69      8652
weighted avg       0.88      0.82      0.84      8652

Category:  direct_report
              precision    recall  f1-score   support

           0       0.98      0.86      0.92      7877
           1       0.36      0.78      0.49       775

    accuracy                           0.85      8652
   macro avg       0.67      0.82      0.70      8652
weig

/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/yingjie/anaconda3/lib/python3.6/site-packages/

              precision    recall  f1-score   support

           0       0.99      0.98      0.98      7946
           1       0.79      0.88      0.83       706

    accuracy                           0.97      8652
   macro avg       0.89      0.93      0.91      8652
weighted avg       0.97      0.97      0.97      8652

Category:  floods
              precision    recall  f1-score   support

           0       0.99      0.96      0.97      8282
           1       0.46      0.87      0.61       370

    accuracy                           0.95      8652
   macro avg       0.73      0.91      0.79      8652
weighted avg       0.97      0.95      0.96      8652

Category:  security
              precision    recall  f1-score   support

           0       1.00      0.98      0.99      8650
           1       0.00      0.00      0.00         2

    accuracy                           0.98      8652
   macro avg       0.50      0.49      0.50      8652
weighted avg       1.00      0.98   

/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/yingjie/anaconda3/lib/python3.6/site-packages/

              precision    recall  f1-score   support

           0       1.00      0.96      0.98      8652
           1       0.00      0.00      0.00         0

    accuracy                           0.96      8652
   macro avg       0.50      0.48      0.49      8652
weighted avg       1.00      0.96      0.98      8652

Category:  other_weather
              precision    recall  f1-score   support

           0       1.00      0.95      0.97      8644
           1       0.01      0.75      0.03         8

    accuracy                           0.95      8652
   macro avg       0.51      0.85      0.50      8652
weighted avg       1.00      0.95      0.97      8652

Category:  medical_help
              precision    recall  f1-score   support

           0       1.00      0.92      0.96      8573
           1       0.07      0.61      0.12        79

    accuracy                           0.92      8652
   macro avg       0.53      0.77      0.54      8652
weighted avg       0.99  

/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [16]:
print('Overall F1 (micro): ', f1_score(y_pred, y_test, average='micro'))
print('Overall precision (micro): ', precision_score(y_pred, y_test, average='micro'))
print('Overall recall (micro): ', recall_score(y_pred, y_test, average='micro'))
print('Overall accuracy: ', (y_pred == y_test.values).mean())

Overall F1 (micro):  0.6424616742031543
Overall precision (micro):  0.5320400409177262
Overall recall (micro):  0.8107220397483716
Overall accuracy:  0.947960009246417


### 6. Improve your model
Use grid search to find better parameters. 

In [17]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x7f9725e82158>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x7f9725e82158>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,
 'tfidf_

In [18]:
parameters = {
    'vect__binary': (True, False),
    'clf__estimator__n_estimators': (100, 500),
    #'clf__estimator__class_weight': [None, 'balanced', 'balanced_subsample'],
}

model = GridSearchCV(pipeline, param_grid=parameters)

In [19]:
%time
model.fit(X_train,y_train)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.58 µs


GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x7f9725e82158>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'clf__estimator__n_estimators': (100, 500),
                         'vect__binary': (True, False)})

In [20]:
model.best_params_

{'clf__estimator__n_estimators': 500, 'vect__binary': False}

In [21]:
y_pred = model.predict(X_test)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [22]:
print('Overall F1 (micro): ', f1_score(y_pred, y_test, average='micro'))
print('Overall precision (micro): ', precision_score(y_pred, y_test, average='micro'))
print('Overall recall (micro): ', recall_score(y_pred, y_test, average='micro'))
print('Overall accuracy: ', (y_pred == y_test.values).mean())

Overall F1 (micro):  0.6441261503829091
Overall precision (micro):  0.5331360514394271
Overall recall (micro):  0.8134790122080383
Overall accuracy:  0.9482296964093081


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### Add genre as one of the feature

In [23]:
df = pd.concat([pd.get_dummies(df['genre']), df], axis=1)

In [24]:
df.drop(['genre'], axis=1, inplace=True)

In [25]:
df.head()

,direct,news,social,id,message,original,related,request,offer,aid_related,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,1,0,0,1,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
X = df[['direct', 'news', 'social', 'message']]
y = df[Y]

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

#### Define ItemSelector Class to add ['direct', 'news', 'social', 'message'] in the pipeline
https://scikit-learn.org/0.18/auto_examples/hetero_feature_union.html

In [28]:
class ItemSelector(BaseEstimator, TransformerMixin):
    """For data grouped by feature, select subset of data at a provided key.

    The data is expected to be stored in a 2D data structure, where the first
    index is over features and the second is over samples.  i.e.

    >> len(data[key]) == n_samples

    Please note that this is the opposite convention to scikit-learn feature
    matrixes (where the first index corresponds to sample).

    ItemSelector only requires that the collection implement getitem
    (data[key]).  Examples include: a dict of lists, 2D numpy array, Pandas
    DataFrame, numpy record array, etc.

    >> data = {'a': [1, 5, 2, 5, 2, 8],
               'b': [9, 4, 1, 4, 1, 3]}
    >> ds = ItemSelector(key='a')
    >> data['a'] == ds.transform(data)

    ItemSelector is not designed to handle data grouped by sample.  (e.g. a
    list of dicts).  If your data is structured this way, consider a
    transformer along the lines of `sklearn.feature_extraction.DictVectorizer`.

    Parameters
    ----------
    key : hashable, required
        The key corresponding to the desired value in a mappable.
    """
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key]

In [29]:
pipeline = Pipeline([
    ('features', FeatureUnion([
        ('text_pipeline', Pipeline([
            ('selector', ItemSelector(key='message')),
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),
        ('genre_enc', ItemSelector(key=['direct', 'social', 'news'])),
    ])),
        ('clf', MultiOutputClassifier(RandomForestClassifier())),
    ])

In [31]:
pipeline.fit(X_train,y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('text_pipeline',
                                                 Pipeline(steps=[('selector',
                                                                  ItemSelector(key='message')),
                                                                 ('vect',
                                                                  CountVectorizer(tokenizer=<function tokenize at 0x7f9725e82158>)),
                                                                 ('tfidf',
                                                                  TfidfTransformer())])),
                                                ('genre_enc',
                                                 ItemSelector(key=['direct',
                                                                   'social',
                                                                   'news']))])),
                ('clf',
                 MultiOutputClassifier

In [32]:
y_pred = pipeline.predict(X_test)

In [33]:
for i in range(len(Y)):
    print(y_test.columns[i])
    print(classification_report(pd.DataFrame(y_pred).iloc[:, i], pd.DataFrame(y_test).iloc[:, i]))

missing_people
              precision    recall  f1-score   support

           0       1.00      0.99      0.99      8652
           1       0.00      0.00      0.00         0

    accuracy                           0.99      8652
   macro avg       0.50      0.49      0.50      8652
weighted avg       1.00      0.99      0.99      8652

related
              precision    recall  f1-score   support

           0       0.36      0.72      0.48      1029
           1       0.96      0.83      0.89      7623

    accuracy                           0.81      8652
   macro avg       0.66      0.77      0.68      8652
weighted avg       0.88      0.81      0.84      8652

direct_report
              precision    recall  f1-score   support

           0       0.97      0.87      0.92      7799
           1       0.39      0.77      0.52       853

    accuracy                           0.86      8652
   macro avg       0.68      0.82      0.72      8652
weighted avg       0.92      0.86    

/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/yingjie/anaconda3/lib/python3.6/site-packages/

              precision    recall  f1-score   support

           0       0.99      0.98      0.98      7960
           1       0.77      0.88      0.82       692

    accuracy                           0.97      8652
   macro avg       0.88      0.93      0.90      8652
weighted avg       0.97      0.97      0.97      8652

floods
              precision    recall  f1-score   support

           0       0.99      0.96      0.98      8273
           1       0.48      0.89      0.63       379

    accuracy                           0.95      8652
   macro avg       0.74      0.92      0.80      8652
weighted avg       0.97      0.95      0.96      8652

security
              precision    recall  f1-score   support

           0       1.00      0.98      0.99      8648
           1       0.00      0.00      0.00         4

    accuracy                           0.98      8652
   macro avg       0.50      0.49      0.50      8652
weighted avg       1.00      0.98      0.99      8652

ref

/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/yingjie/anaconda3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/yingjie/anaconda3/lib/python3.6/site-packages/

In [34]:
print('Overall F1 (micro): ', f1_score(y_pred, y_test, average='micro'))
print('Overall precision (micro): ', precision_score(y_pred, y_test, average='micro'))
print('Overall recall (micro): ', recall_score(y_pred, y_test, average='micro'))
print('Overall accuracy: ', (y_pred == y_test.values).mean())

Overall F1 (micro):  0.6424186210856412
Overall precision (micro):  0.5313824346047056
Overall recall (micro):  0.8121161362367393
Overall accuracy:  0.9480145887912879


### Try out XGBoost model

In [35]:
pipeline = Pipeline([
    ('features', FeatureUnion([
        ('text_pipeline', Pipeline([
            ('selector', ItemSelector(key='message')),
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),
        ('genre_enc', ItemSelector(key=['direct', 'social', 'news'])),
    ])),
        ('clf', MultiOutputClassifier(XGBClassifier())),
    ])

In [36]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('text_pipeline',
                                                 Pipeline(steps=[('selector',
                                                                  ItemSelector(key='message')),
                                                                 ('vect',
                                                                  CountVectorizer(tokenizer=<function tokenize at 0x7f9725e82158>)),
                                                                 ('tfidf',
                                                                  TfidfTransformer())])),
                                                ('genre_enc',
                                                 ItemSelector(key=['direct',
                                                                   'social',
                                                                   'news']))])),
                ('clf', MultiOutputClassifier(estimator=XGBCla

In [37]:
y_pred = pipeline.predict(X_test)

In [38]:
print('Overall F1 (micro): ', f1_score(y_pred, y_test, average='micro'))
print('Overall precision (micro): ', precision_score(y_pred, y_test, average='micro'))
print('Overall recall (micro): ', recall_score(y_pred, y_test, average='micro'))
print('Overall accuracy: ', (y_pred == y_test.values).mean())

Overall F1 (micro):  0.6599871822260202
Overall precision (micro):  0.5643358176238492
Overall recall (micro):  0.7946805226875193
Overall accuracy:  0.9489007037550727


In [39]:
pipeline.get_params()

{'memory': None,
 'steps': [('features',
   FeatureUnion(transformer_list=[('text_pipeline',
                                   Pipeline(steps=[('selector',
                                                    ItemSelector(key='message')),
                                                   ('vect',
                                                    CountVectorizer(tokenizer=<function tokenize at 0x7f9725e82158>)),
                                                   ('tfidf',
                                                    TfidfTransformer())])),
                                  ('genre_enc',
                                   ItemSelector(key=['direct', 'social',
                                                     'news']))])),
  ('clf', MultiOutputClassifier(estimator=XGBClassifier()))],
 'verbose': False,
 'features': FeatureUnion(transformer_list=[('text_pipeline',
                                 Pipeline(steps=[('selector',
                                                  It

In [40]:
parameters = {
    'clf__estimator__max_depth': (7, 10),
    'clf__estimator__colsample_bytree': (0.6, 1)
}

model = GridSearchCV(pipeline, param_grid=parameters)

In [41]:
model.fit(X_train,y_train)

GridSearchCV(estimator=Pipeline(steps=[('features',
                                        FeatureUnion(transformer_list=[('text_pipeline',
                                                                        Pipeline(steps=[('selector',
                                                                                         ItemSelector(key='message')),
                                                                                        ('vect',
                                                                                         CountVectorizer(tokenizer=<function tokenize at 0x7f9725e82158>)),
                                                                                        ('tfidf',
                                                                                         TfidfTransformer())])),
                                                                       ('genre_enc',
                                                                        ItemSelector(key=[

In [42]:
model.best_params_

{'clf__estimator__colsample_bytree': 0.6, 'clf__estimator__max_depth': 10}

In [43]:
y_pred = model.predict(X_test)

In [44]:
print('Overall F1 (micro): ', f1_score(y_pred, y_test, average='micro'))
print('Overall precision (micro): ', precision_score(y_pred, y_test, average='micro'))
print('Overall recall (micro): ', recall_score(y_pred, y_test, average='micro'))
print('Overall accuracy: ', (y_pred == y_test.values).mean())

Overall F1 (micro):  0.6795075789297379
Overall precision (micro):  0.5928686248721321
Overall recall (micro):  0.7958022754021185
Overall accuracy:  0.9508527251245698


### 9. Export your model as a pickle file

In [46]:
with open('models/classifier.pkl', 'wb') as f:
        pickle.dump(model, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.